<a href="https://colab.research.google.com/github/AbR04/5A-AI/blob/main/WEEK_7_AI_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
class Pair:
    """Represents a pair (or cons cell) used in lists."""
    def __init__(self, first, second):
        self.first = first
        self.second = second

    def __repr__(self):
        return f"({self.first} . {self.second})" if self.second is not None else f"({self.first})"


nil = None


class Frame:
    """Represents an environment that maps variables to their values."""
    def __init__(self, parent=None):
        self.parent = parent
        self.bindings = {}

    def define(self, var, value):
        """Binds a variable to a value in this frame."""
        self.bindings[var] = value

    def lookup(self, var):
        """Returns the value of a variable, looking it up in the current frame and its ancestors."""
        if var in self.bindings:
            return self.bindings[var]
        elif self.parent is not None:
            return self.parent.lookup(var)
        else:
            return var


def isvar(expr):
    """Returns True if the expression is a variable (starts with ?)."""
    return isinstance(expr, str) and expr.startswith('?')


def scheme_atomp(expr):
    """Checks if the expression is an atom (not a pair)."""
    return not isinstance(expr, Pair)


def lookup(expr, env):
    """Looks up an expression in the environment, applying bindings."""
    if isvar(expr):
        return env.lookup(expr)
    return expr


def unify(e, f, env):
    """Unifies two expressions (e, f) and modifies the environment."""
    e = lookup(e, env)
    f = lookup(f, env)
    if e == f:
        return True
    elif isvar(e):
        env.define(e, f)
        return True
    elif isvar(f):
        env.define(f, e)
        return True
    elif scheme_atomp(e) or scheme_atomp(f):
        return False
    else:
        return unify(e.first, f.first, env) and unify(e.second, f.second, env)


def rename_variables(expr, n):
    """Renames variables in an expression by appending a unique identifier."""
    if isvar(expr):
        return f"{expr}_{n}"
    elif isinstance(expr, Pair):
        return Pair(rename_variables(expr.first, n), rename_variables(expr.second, n))
    else:
        return expr


def read_line(s):
    """Parses a string into a nested Pair structure."""
    def read_tokens(tokens):
        if not tokens:
            return nil
        first = tokens.pop(0)
        if first == '(':
            sub_expr = read_tokens(tokens)
            return Pair(sub_expr, read_tokens(tokens))
        elif first == ')':
            return nil
        else:
            return Pair(first, read_tokens(tokens))

    tokens = s.strip().replace('(', ' ( ').replace(')', ' ) ').split()
    return read_tokens(tokens)


def search(clauses, env, depth, facts, DEPTH_LIMIT=None):
    """Searches for an application of rules to establish all the CLAUSES, non-destructively extending ENV."""
    if clauses is nil:
        yield env
    elif DEPTH_LIMIT is None or depth <= DEPTH_LIMIT:
        if isinstance(clauses.first, Pair) and clauses.first.first in ('not', '~'):
            clause = ground(clauses.first.second, env)
            try:
                next(search(clause, env, 0, facts, DEPTH_LIMIT))
            except StopIteration:
                env_head = Frame(env)
                for result in search(clauses.second, env_head, depth + 1, facts, DEPTH_LIMIT):
                    yield result
        else:
            for fact in facts:
                fact = rename_variables(fact, get_unique_id())
                env_head = Frame(env)
                if unify(fact.first, clauses.first, env_head):
                    for env_rule in search(fact.second, env_head, depth + 1, facts, DEPTH_LIMIT):
                        for result in search(clauses.second, env_rule, depth + 1, facts, DEPTH_LIMIT):
                            yield result


def get_unique_id():
    """Generates a unique identifier for renaming variables."""
    from random import randint
    return randint(1, 10000)


def ground(expr, env):
    """Grounds the expression by replacing variables with their bindings in the environment."""
    if isvar(expr):
        return lookup(expr, env)
    elif isinstance(expr, Pair):
        return Pair(ground(expr.first, env), ground(expr.second, env))
    else:
        return expr


# Example facts
facts = [
    Pair(Pair('ancestor', Pair('abraham', Pair('clinton', nil))), nil),
    Pair(Pair('ancestor', Pair('fillmore', Pair('abraham', nil))), nil),
    Pair(Pair('ancestor', Pair('eisenhower', Pair('fillmore', nil))), nil),
    Pair(Pair('dog', Pair(Pair('name', Pair('herbert', nil)), Pair('color', Pair('brown', nil)))), nil),
    Pair(Pair('dog', Pair(Pair('name', Pair('herbert', nil)), Pair('color', Pair('tan', nil)))), nil)
]


# Example query
query = read_line("(?x ?x)")
fact = read_line("((a ?y c) (a b ?z))")

# Initialize environment
env = Frame(None)

# Perform unification
print("Unify:", unify(fact, query, env))
print("Bindings after unification:", env.bindings)

# Search example query
clauses = read_line("((ancestor ?a clinton) (ancestor ?a ?brown-dog) (dog (name ?brown-dog) (color brown)))")
result = search(clauses, Frame(None), 0, facts)
for res in result:
    print("Found bindings:", res.bindings)


Unify: True
Bindings after unification: {'?x': (a . (?y . (c))), '?y': 'b', '?z': 'c'}
